In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data loading

In [ ]:
import cv2
import os
import pandas as pd
import pathlib
import numpy as np
from PIL import Image
from PIL import Image
from torchvision import transforms
from skimage.metrics import structural_similarity as ssim
!pip install lpips
import torch
import torchvision
import torchvision.transforms as transforms
import lpips
from google.colab.patches import cv2_imshow
trans = transforms.ToTensor()
import tensorflow as tf
import glob

In [ ]:
def get_img_paths(predict_dir,original_dir):
  predict_root = pathlib.Path(predict_dir)
  predict_paths = list(sorted(predict_root.rglob("*.png*")))
  predict_paths_lst = [str(path) for path in predict_paths]

  original_root = pathlib.Path(original_dir)
  original_paths = list(sorted(original_root.rglob("*.png*")))
  original_paths_lst = [str(path) for path in original_paths]

  original_array = np.asarray(original_paths_lst)
  predict_array = np.asarray(predict_paths_lst)
  return predict_array, original_array

In [ ]:
def evaluate_final(test_data_path, test_target_path):
  num_test_imgs = len(test_data_path)
  num_losses = 4
  eval_numpy = np.zeros((len(test_data_path),4), dtype=float)
  
  eval_numpy = np.zeros(shape = (num_test_imgs,num_losses))
  for itr in range(num_test_imgs):
    test_data_img = cv2.imread(test_data_path[itr])
    test_data_cvt = cv2.cvtColor(test_data_img, cv2.COLOR_BGR2RGB)

    test_target_img = cv2.imread(test_target_path[itr])
    test_target_cvt = cv2.cvtColor(test_target_img, cv2.COLOR_BGR2RGB)

    if test_data_cvt.shape != test_target_cvt.shape:
      test_data_cvt = cv2.resize(test_data_cvt, (test_target_cvt.shape[1],test_target_cvt.shape[0]))

    psnr, SSIM,Alex_Loss, VGG_Loss = eval_metrics(test_target_cvt, test_data_cvt)
    
    eval_numpy[itr,0],eval_numpy[itr,1],eval_numpy[itr,2],eval_numpy[itr,3] = psnr, SSIM,Alex_Loss, VGG_Loss
    
  
  return eval_numpy

In [ ]:
loss_fn_alex = lpips.LPIPS(net='alex') # best forward scores
loss_fn_vgg = lpips.LPIPS(net='vgg') # closer to "traditional" perceptual loss, when used for optimization

def eval_metrics(img_1, img_2):
  
  # PSNR
  psnr = cv2.PSNR(img_1,img_2)
  #print('psnr:', psnr)
  
  
  # SSIM
  (score,diff) = ssim(img_1, img_2, full = True, multichannel = True)
  # diff = (diff*255)
  SSIM = score
  
  # LPIPS
  im_1 = np.moveaxis(img_1,-1,0)
  img1_torch = torch.tensor(im_1)
  img1 = torch.unsqueeze(img1_torch,0)

  im_2 = np.moveaxis(img_2,-1,0)
  img2_torch = torch.tensor(im_2)
  img2 = torch.unsqueeze(img2_torch,0)

  Alex_Loss = loss_fn_alex(img1, img2)
  VGG_loss = loss_fn_vgg(img1, img2)
  
  return psnr, SSIM, Alex_Loss.item(),VGG_loss.item()

In [ ]:
def losses_of_set(eval_numpy):
  mean = np.mean(eval_numpy,axis =0)
  var = np.var(eval_numpy,axis=0)

  print('PSNR : {:0.4f} +/- {:0.4f}'.format(mean[0],var[0]))
  print('SSIM : {:0.4f} +/- {:0.4f}'.format(mean[1],var[1]))
  print('Alex_loss : {:0.4f} +/- {:0.4f}'.format(mean[2],var[2]))
  print('VGG_loss : {:0.4f} +/- {:0.4f}'.format(mean[3],var[3]))

# Eval

In [ ]:
del original_test, predict_bicubic, array_predict_paths, array_original_paths, eval_numpy

In [ ]:
original_test = 'Path to original images'
predict_bicubic = 'Path to predicted images'
array_predict_paths, array_original_paths = get_img_paths(predict_bicubic,original_test)
eval_numpy = evaluate_final(array_predict_paths, array_original_paths)
losses_of_set(eval_numpy)

In [ ]:
df = pd.DataFrame(eval_numpy, columns=['PSNR', 'SSIM', 'ALEX','VGG'])
df.to_csv('result.csv')